# 4.1.6-使用ArcGIS-Pro和Arcy制作建筑年代预测结果在150米精度范围的准确度

我们要复现论文中的图8：

图片上表现的是预测的精准程度在150m的网格上的空间分布，图中可以看出：市中心的错误率高于郊区。这可能是由于建筑年龄的高度多样性、市中心旧建筑的频繁改造以及上述对改造建筑的严格规定所致。阿姆斯特丹郊区没有明显的空间格局，这表明分类结果的空间相关性很小。为了证明空间相关性小，作者还计算了莫兰指数，城市郊区结果的 Moran's I 为 0.27。 Moran's I 的范围为 -1 到 1，其中 -1 表示完美分散`dispersion` ，0 表示完美随机性  `randomness`，1 表示完美聚类`clustering`。我们预测的阿姆斯特丹郊区的莫兰指数为0.27，建筑年代表现出较弱的空间自相关性。

## 1.获取含建筑位置信息和预测准确度的建筑数据


In [1]:
import geopandas as gpd
import pandas as pd

gdb = r"../../5-ArcgisPro工程/建筑风格和年代深度学习.gdb" # 
lr_name = 'predictions_with_building_age_diff_city_wide' # 在4.1.4中保存的全市范围建筑准确度的结果
gdf = gpd.read_file(filename=gdb, layer=lr_name)
gdf.head()

,identificatie,id,true_label,diff,pre_label,geometry
0,0363100012061709,None,None,NaN,None,"MULTIPOLYGON (((635512.808 5802343.021, 635518..."
1,0363100012061225,None,None,NaN,None,"MULTIPOLYGON (((628083.847 5808893.547, 628087..."
2,0363100012061228,0363100012061228,–2023,0.0,–2023,"MULTIPOLYGON (((635718.103 5802400.059, 635709..."
3,0363100012062224,None,None,NaN,None,"MULTIPOLYGON (((632084.702 5803319.551, 632084..."
4,0363100012063200,None,None,NaN,None,"MULTIPOLYGON (((628012.652 5798762.544, 628013..."


## 2.利用ArcPy的创建渔网工具创建150米的空间网格

### 2.1创建渔网函数解析


1. 官方帮助文档：[创建渔网 (数据管理)](https://pro.arcgis.com/zh-cn/pro-app/3.0/tool-reference/data-management/create-fishnet.htm)
2. 在[《利用ArcGIS_Python制作考虑路况的交通等时圈》](https://cdn.renhai-lab.tech/archives/4.2.14-%E5%AE%9E%E6%93%8D3-%E5%88%A9%E7%94%A8ArcGIS_Python%E5%88%B6%E4%BD%9C%E8%80%83%E8%99%91%E8%B7%AF%E5%86%B5%E7%9A%84%E4%BA%A4%E9%80%9A%E7%AD%89%E6%97%B6%E5%9C%88#3.%E5%88%9B%E5%BB%BA%E6%B8%94%E7%BD%91)文也讲过：点击文章查看, 本文中的代码是在此基础上修改的。关键信息如下：
- **a.创建渔网函数的参数**：
    - out_feature_class：包含由矩形像元组成的渔网的输出要素类。
    - **origin_coord**：矩形框的左下端点为原点
    - **y_axis_coord**：此点与原点的连线用于判断旋转的角度 我们不用旋转所以定义为原点正上方的点
    - cell_width：网格的宽度
    - cell_height：网格的高度
    - number_rows：填'0',代表留空，由cell_width和cell_height决定
    - number_columns：填'0'，由cell_width和cell_height决定
    - corner_coord：填'0'
    - **labels**：'LABELS'
    - **template**：以空格分隔的坐标字符串 - 将使用指定渔网的范围。坐标以 x-min，y-min，x-max，y-max 的顺序表示。
    - geometry_type：生成面
- **b.创建渔网返回的结果**：
    - out_feature_class：包含由矩形像元组成的渔网的输出要素类。
    - out_label：创建一个新的点要素类，其中每个渔网像元中心都具有标注点。如果选中了创建标注点参数（Python 中的 labels = 'LABELS'），则会创建一个新的点要素类，其中每个渔网像元中心都具有标注点。此要素类的名称以 _label 为后缀并与输出要素类相同，且创建于同一位置。

### 2.2 定义origin_coord和y_axis_coord

从上文(4.1.5-使用ArcGIS-Pro和Arcy制作建筑年代预测结果在150米精度范围的准确度.ipynb)gdf中获取边界。

In [2]:
from shapely.geometry import box

# total_bounds返回一个元组，包含最小的x，最小的y，最大的x，最大的y
x_min, y_min, x_max, y_max = gdf.total_bounds
# 创建一个边界框
bbox_geometry = box(x_min, y_min, x_max, y_max)
# 创建一个GeoDataFrame，将边界框作为几何图形
gdf_bbox = gpd.GeoDataFrame({'geometry': bbox_geometry}, index=[0]).set_crs(32631)
gdf_bbox

,geometry
0,"POLYGON ((640252.432 5798425.740, 640252.432 5..."


In [3]:
# # (可选)将边界框保存为shapefile
# gdf_bbox.to_file(filename=gdb, layer='city_wide_bbox')

In [4]:
# 定义origin_coord和y_axis_coord 空格分割
origin_coord = f"{x_min}  {y_min}"
y_axis_coord = f"{x_min}  {y_max}"
origin_coord, y_axis_coord

('617784.8387999535  5798425.739600182',
 '617784.8387999535  5810551.856800079')

### 导入arcpy创建渔网

In [5]:
# 导入arcpy并设置工作空间
import arcpy

# 设置工作空间
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True

In [6]:
scales = 150 # 网格的单元尺度 单位为米

In [7]:
extent = f"{x_min} {y_min} {x_max} {y_max}"
extent

'617784.8387999535 5798425.739600182 640252.4324002266 5810551.856800079'

In [8]:
# 设置空间参考对象
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(32631)

In [9]:
out_fcs = 'fishnet' # 有bug 不能数字开头

In [10]:
# 设置工作空间
arcpy.management.CreateFishnet(out_feature_class = out_fcs, # 包含由矩形像元组成的渔网的输出要素类。
                               origin_coord = origin_coord, # 矩形框的左下端点为原点
                               y_axis_coord = y_axis_coord, # 此点与原点的连线用于判断旋转的角度 我们不用旋转所以定义为原点正上方的点
                               cell_width = scales, 
                               cell_height = scales,
                               number_rows = "0", # 留空，由cell_width和cell_height决定
                               number_columns = "0", # 留空，由cell_width和cell_height决定
                               corner_coord = None, # 对角坐标不填写
                               labels = "LABELS", 
                               template = extent, # 以空格分隔的坐标字符串 - 将使用指定渔网的范围。坐标以 x-min，y-min，x-max，y-max 的顺序表示。
                               geometry_type = "POLYGON" # 生成面
                               )

<Result '../../5-ArcgisPro工程/建筑风格和年代深度学习.gdb\\fishnet'>

In [12]:
# 定义新产生的点要素的名称
out_label = out_fcs + "_label"

## 3.空间连接

geopandas和arcpy都有空间连接的功能，但是geopandas的空间连接功能更强大，而且方便进行数据统计，所以我们使用geopandas的空间连接功能。

在选择处理工具的时候，我不喜欢用arcpy的游标进行数据处理，因为arcpy的游标处理数据的速度太慢了，而且代码也不够简洁。所以我更喜欢用geopandas基于dataframe的处理，更强大也更简洁。
但是arcpy中提供一些实用的工具，比如同时创建渔网和渔网标注点，这个功能在geopandas中没有，所以我们用arcpy创建渔网，然后用geopandas进行空间连接。另外，制图的时候，我们也会用到ArcGIS Pro，可以实时看到制图的效果，并且渲染效果更好。

In [13]:
# 读取渔网
gdf_fishnet = gpd.read_file(filename=gdb, layer=out_fcs)

# 读取渔网标注点
gdf_fishnet_label = gpd.read_file(filename=gdb, layer=out_label)

In [14]:
len(gdf_fishnet), len(gdf_fishnet_label)

(12150, 12150)

In [15]:
# 给渔网和标注点添加id列
gdf_fishnet['id'] = gdf_fishnet.index
gdf_fishnet_label['id'] = gdf_fishnet_label.index

In [16]:
# 查看geopandas的版本
gpd.__version__

'0.14.0'

### 空间连接的方法：一种是使用geopandas.GeoDataFrame.sjoin_nearest 空间连接，一种是使用geopandas.GeoDataFrame.sjoin 空间连接

- [官方帮助文档](https://geopandas.org/en/latest/docs/user_guide/mergingdata.html#spatial-joins)


关键参数：
1. predicate 值，这些值在 [Shapely文档](http://shapely.readthedocs.io/en/latest/manual.html#binary-predicates) 中定义：

    intersects 相交
    
    contains 包含
    
    within 之内
    
    touches 触摸
    
    crosses 十字架
    
    overlaps 重叠
how：
2. how 参数指定将发生的连接类型以及在结果 GeoDataFrame 中保留哪些几何图形。它接受以下选项：
    left ：使用您提供给 GeoDataFrame.sjoin() 的第一个（或 left_df） GeoDataFrame 中的索引；仅保留 left_df 几何列
    
    right ：使用第二个（或 right_df）的索引；仅保留 right_df 几何列
    
    inner ：使用两个 GeoDataFrame 索引值的交集；仅保留 left_df 几何列

In [17]:
# 我们已经有方格网了，直接使用geopandas.GeoDataFrame.sjoin 空间连接, 保留左边的数据，即渔网的数据，然后根据右侧的数据进行计算
gdf_fishnet = gpd.sjoin(gdf_fishnet, gdf, how='left', predicate='intersects') # op参数在将来的版本中将被弃用，并建议您使用predicate参数代替。
len(gdf_fishnet)

197114

In [19]:
gdf_fishnet.head()

,Shape_Length,Shape_Area,geometry,id_left,index_right,identificatie,id_right,true_label,diff,pre_label
0,600.0,22500.0,"MULTIPOLYGON (((617784.839 5798425.740, 617784...",0,NaN,NaN,NaN,NaN,NaN,NaN
1,600.0,22500.0,"MULTIPOLYGON (((617934.839 5798425.740, 617934...",1,NaN,NaN,NaN,NaN,NaN,NaN
2,600.0,22500.0,"MULTIPOLYGON (((618084.839 5798425.740, 618084...",2,NaN,NaN,NaN,NaN,NaN,NaN
3,600.0,22500.0,"MULTIPOLYGON (((618234.839 5798425.740, 618234...",3,NaN,NaN,NaN,NaN,NaN,NaN
4,600.0,22500.0,"MULTIPOLYGON (((618384.839 5798425.740, 618384...",4,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# 扔掉空网格数据
gdf_fishnet = gdf_fishnet.dropna(subset=['id_right'])
gdf_fishnet.head()

,Shape_Length,Shape_Area,geometry,id_left,index_right,identificatie,id_right,true_label,diff,pre_label
58,600.0,22500.0,"MULTIPOLYGON (((626484.839 5798425.740, 626484...",58,78258.0,0363100012185957,0363100012185957,-1944,3.0,–2023
60,600.0,22500.0,"MULTIPOLYGON (((626784.839 5798425.740, 626784...",60,76728.0,0363100012075481,0363100012075481,–2023,0.0,–2023
208,600.0,22500.0,"MULTIPOLYGON (((626484.839 5798575.740, 626484...",208,77055.0,0363100012092829,0363100012092829,–1995,0.0,–1995
209,600.0,22500.0,"MULTIPOLYGON (((626634.839 5798575.740, 626634...",209,77055.0,0363100012092829,0363100012092829,–1995,0.0,–1995
209,600.0,22500.0,"MULTIPOLYGON (((626634.839 5798575.740, 626634...",209,77114.0,0363100012096189,0363100012096189,-1978,0.0,-1978


In [21]:
# 计算预测的准确度 
# 我们对diff列取绝对值，然后正则化到0-1
gdf_fishnet['diff'].abs() # 取绝对值

58       3.0
60       0.0
208      0.0
209      0.0
209      0.0
        ... 
11473    0.0
11473    0.0
11474    1.0
11474    1.0
11474    1.0
Name: diff, Length: 92666, dtype: float64

In [22]:
gdf_fishnet['diff'].abs().max() # 取绝对值的最大值

8.0

In [23]:
gdf_fishnet['diff'].abs() / gdf_fishnet['diff'].abs().max() # 取绝对值的最大值

58       0.375
60       0.000
208      0.000
209      0.000
209      0.000
         ...  
11473    0.000
11473    0.000
11474    0.125
11474    0.125
11474    0.125
Name: diff, Length: 92666, dtype: float64

In [24]:
# 取值0-1 diff是差值为0预测准确，为1预测不准确，但是我们需要的是预测准确度，所以取1-diff
gdf_fishnet['accuracy'] = 1 - gdf_fishnet['diff'].abs() / gdf_fishnet['diff'].abs().max()

In [25]:
gdf_fishnet['accuracy']

58       0.625
60       1.000
208      1.000
209      1.000
209      1.000
         ...  
11473    1.000
11473    1.000
11474    0.875
11474    0.875
11474    0.875
Name: accuracy, Length: 92666, dtype: float64

In [32]:
accuracy_150m = gdf_fishnet.dissolve(by='id_left', aggfunc={"accuracy": "mean"})
accuracy_150m

,geometry,accuracy
id_left,,
58,"POLYGON ((626484.839 5798425.740, 626484.839 5...",0.625
60,"POLYGON ((626784.839 5798425.740, 626784.839 5...",1.000
208,"POLYGON ((626484.839 5798575.740, 626484.839 5...",1.000
209,"POLYGON ((626634.839 5798725.740, 626784.839 5...",1.000
210,"POLYGON ((626784.839 5798575.740, 626784.839 5...",1.000
...,...,...
11470,"POLYGON ((628434.839 5809975.740, 628434.839 5...",1.000
11471,"POLYGON ((628584.839 5809825.740, 628434.839 5...",1.000
11472,"POLYGON ((628584.839 5809825.740, 628584.839 5...",1.000


In [33]:
cols_to_keep = ['identificatie', 'true_label', 'pre_label', 'diff']

# 使用dissolve进行空间聚合并计算accuracy的平均值
accuracy_150m = gdf_fishnet.dissolve(by='id_left', aggfunc={"accuracy": "mean"})

# 直接使用join来连接其他需要的列
accuracy_150m = accuracy_150m.join(gdf_fishnet.set_index('id_left')[cols_to_keep])
accuracy_150m

,geometry,accuracy,identificatie,true_label,pre_label,diff
id_left,,,,,,
58,"POLYGON ((626484.839 5798425.740, 626484.839 5...",0.625,0363100012185957,-1944,–2023,3.0
60,"POLYGON ((626784.839 5798425.740, 626784.839 5...",1.000,0363100012075481,–2023,–2023,0.0
208,"POLYGON ((626484.839 5798575.740, 626484.839 5...",1.000,0363100012092829,–1995,–1995,0.0
209,"POLYGON ((626634.839 5798725.740, 626784.839 5...",1.000,0363100012092829,–1995,–1995,0.0
209,"POLYGON ((626634.839 5798725.740, 626784.839 5...",1.000,0363100012096189,-1978,-1978,0.0
...,...,...,...,...,...,...
11473,"POLYGON ((628734.839 5809825.740, 628734.839 5...",1.000,0363100012108160,–1995,–1995,0.0
11473,"POLYGON ((628734.839 5809825.740, 628734.839 5...",1.000,0363100012106234,–1995,–1995,0.0
11474,"POLYGON ((628884.839 5809975.740, 629034.839 5...",0.875,0363100012063965,–1995,–2023,1.0


In [34]:
# 我们需要的是点的准确度，所以将渔网的准确度赋值给渔网标注点 并且需要扔掉空值 我们使用merge
gdf_fishnet_point = gdf_fishnet_label.merge(accuracy_150m, left_on='id',right_on='id_left' , how='right')
gdf_fishnet_point

,geometry_x,id,geometry_y,accuracy,identificatie,true_label,pre_label,diff
0,POINT (626559.839 5798500.740),58,"POLYGON ((626484.839 5798425.740, 626484.839 5...",0.625,0363100012185957,-1944,–2023,3.0
1,POINT (626859.839 5798500.740),60,"POLYGON ((626784.839 5798425.740, 626784.839 5...",1.000,0363100012075481,–2023,–2023,0.0
2,POINT (626559.839 5798650.740),208,"POLYGON ((626484.839 5798575.740, 626484.839 5...",1.000,0363100012092829,–1995,–1995,0.0
3,POINT (626709.839 5798650.740),209,"POLYGON ((626634.839 5798725.740, 626784.839 5...",1.000,0363100012092829,–1995,–1995,0.0
4,POINT (626709.839 5798650.740),209,"POLYGON ((626634.839 5798725.740, 626784.839 5...",1.000,0363100012096189,-1978,-1978,0.0
...,...,...,...,...,...,...,...,...
92661,POINT (628809.839 5809900.740),11473,"POLYGON ((628734.839 5809825.740, 628734.839 5...",1.000,0363100012108160,–1995,–1995,0.0
92662,POINT (628809.839 5809900.740),11473,"POLYGON ((628734.839 5809825.740, 628734.839 5...",1.000,0363100012106234,–1995,–1995,0.0
92663,POINT (628959.839 5809900.740),11474,"POLYGON ((628884.839 5809975.740, 629034.839 5...",0.875,0363100012063965,–1995,–2023,1.0
92664,POINT (628959.839 5809900.740),11474,"POLYGON ((628884.839 5809975.740, 629034.839 5...",0.875,0363100012073264,–1995,–2023,1.0


In [37]:
## 清理数据
accuracy_polygon = gdf_fishnet_point.drop(columns=['id', 'geometry_x']).rename(columns={'geometry_y': 'geometry'})
accuracy_polygon

,geometry,accuracy,identificatie,true_label,pre_label,diff
0,"POLYGON ((626484.839 5798425.740, 626484.839 5...",0.625,0363100012185957,-1944,–2023,3.0
1,"POLYGON ((626784.839 5798425.740, 626784.839 5...",1.000,0363100012075481,–2023,–2023,0.0
2,"POLYGON ((626484.839 5798575.740, 626484.839 5...",1.000,0363100012092829,–1995,–1995,0.0
3,"POLYGON ((626634.839 5798725.740, 626784.839 5...",1.000,0363100012092829,–1995,–1995,0.0
4,"POLYGON ((626634.839 5798725.740, 626784.839 5...",1.000,0363100012096189,-1978,-1978,0.0
...,...,...,...,...,...,...
92661,"POLYGON ((628734.839 5809825.740, 628734.839 5...",1.000,0363100012108160,–1995,–1995,0.0
92662,"POLYGON ((628734.839 5809825.740, 628734.839 5...",1.000,0363100012106234,–1995,–1995,0.0
92663,"POLYGON ((628884.839 5809975.740, 629034.839 5...",0.875,0363100012063965,–1995,–2023,1.0
92664,"POLYGON ((628884.839 5809975.740, 629034.839 5...",0.875,0363100012073264,–1995,–2023,1.0


In [38]:
accuracy_polygon_gdf = gpd.GeoDataFrame(accuracy_polygon, geometry='geometry').set_crs(32631)
accuracy_polygon_gdf

,geometry,accuracy,identificatie,true_label,pre_label,diff
0,"POLYGON ((626484.839 5798425.740, 626484.839 5...",0.625,0363100012185957,-1944,–2023,3.0
1,"POLYGON ((626784.839 5798425.740, 626784.839 5...",1.000,0363100012075481,–2023,–2023,0.0
2,"POLYGON ((626484.839 5798575.740, 626484.839 5...",1.000,0363100012092829,–1995,–1995,0.0
3,"POLYGON ((626634.839 5798725.740, 626784.839 5...",1.000,0363100012092829,–1995,–1995,0.0
4,"POLYGON ((626634.839 5798725.740, 626784.839 5...",1.000,0363100012096189,-1978,-1978,0.0
...,...,...,...,...,...,...
92661,"POLYGON ((628734.839 5809825.740, 628734.839 5...",1.000,0363100012108160,–1995,–1995,0.0
92662,"POLYGON ((628734.839 5809825.740, 628734.839 5...",1.000,0363100012106234,–1995,–1995,0.0
92663,"POLYGON ((628884.839 5809975.740, 629034.839 5...",0.875,0363100012063965,–1995,–2023,1.0
92664,"POLYGON ((628884.839 5809975.740, 629034.839 5...",0.875,0363100012073264,–1995,–2023,1.0


In [39]:
# 保存到保存面数据到数据库 在ArcGIS Pro制图
accuracy_polygon_gdf.to_file(filename=gdb, layer='accuracy_150m_polygon')

<ipython-input-39-c92a42c7ae6e>:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  accuracy_polygon_gdf.to_file(filename=gdb, layer='accuracy_150m_polygon')


In [40]:
# 保存一份geojson 用于空间自相关研究
accuracy_polygon_gdf.to_file(filename='../../data/output/accuracy_150m_polygon.geojson')

In [42]:
### 保存点数据
accuracy_points = gdf_fishnet_point.drop(columns=['id',  'geometry_y']).rename(
    columns={'geometry_x': 'geometry'})
gpd.GeoDataFrame(accuracy_points, geometry='geometry').set_crs(32631).to_file(filename=gdb, layer='accuracy_150m_points')

<ipython-input-42-0e847e309733>:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(accuracy_points, geometry='geometry').set_crs(32631).to_file(filename=gdb, layer='accuracy_150m_points')


## ArcGIS Pro制图
详见ArcGIS Pro工程

![](../../5-ArcgisPro工程/阿姆斯特丹全市范围建筑年代预测准确度空间分布图.jpg)